In [2]:
import pandas as pd
import numpy as np

In [ ]:
data_dir = "C:/Users/16306/Downloads/"

In [12]:
#Read in data
df = pd.read_csv("C:/Users/16306/Downloads/application_train.csv")
df.head()

#Pull target variable and remove from predictors
target_var = df["TARGET"]
df.drop(columns=["SK_ID_CURR", "TARGET"], inplace=True)

#Drop NA
df.dropna(inplace = True)

In [33]:
#Zero binary columns
count = 0
col_list = []
for col in df.columns:
    if len(df[col].value_counts()) == 2:
        if df[col].dtype != np.int64:
            col_list.append(col)
            count += 1
        
col_list

['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_REALTY', 'EMERGENCYSTATE_MODE']

In [50]:
new_vars = pd.get_dummies(df[col_list])[["NAME_CONTRACT_TYPE_Cash loans", "CODE_GENDER_F", "FLAG_OWN_REALTY_N", "EMERGENCYSTATE_MODE_No"]]

,NAME_CONTRACT_TYPE_Cash loans,CODE_GENDER_F,FLAG_OWN_REALTY_N,EMERGENCYSTATE_MODE_No
71,1,0,0,1
124,1,1,0,1
152,1,1,1,1
161,1,0,1,1
255,1,0,1,1
...,...,...,...,...
307358,1,1,0,1
307359,1,1,0,1
307407,1,1,0,1
307456,1,1,0,1


In [ ]:
df["CODE_GENDER"].value_counts()

In [ ]:
df["FLAG_OWN_REALITY"].value_counts()

In [11]:
df_ = pd.get_dummies(df)
df_corrs = df_.corr()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
SK_ID_CURR,1.000000,-0.002108,-0.001129,-0.001820,-0.000343,-0.000433,-0.000232,0.000849,-0.001500,0.001366,...,0.000553,0.002073,-0.000976,0.000072,-0.001396,0.002300,-0.001281,-0.000270,0.000510,0.002549
TARGET,-0.002108,1.000000,0.019187,-0.003982,-0.030369,-0.012817,-0.039645,-0.037227,0.078239,-0.044932,...,0.000982,-0.006777,-0.001713,-0.009384,0.000628,-0.033119,-0.012657,0.007946,-0.042201,0.004829
CNT_CHILDREN,-0.001129,0.019187,1.000000,0.012882,0.002145,0.021374,-0.001827,-0.025573,0.330938,-0.239818,...,-0.001830,-0.005272,-0.000709,0.001607,-0.002032,-0.020892,-0.025088,0.011036,-0.038644,0.004525
AMT_INCOME_TOTAL,-0.001820,-0.003982,0.012882,1.000000,0.156870,0.191657,0.159610,0.074796,0.027261,-0.064223,...,0.000718,0.011696,0.006149,0.023886,0.003886,0.032753,0.016523,-0.003369,0.050174,-0.002894
AMT_CREDIT,-0.000343,-0.030369,0.002145,0.156870,1.000000,0.770138,0.986968,0.099738,-0.055436,-0.066838,...,0.000869,0.014314,0.007987,0.027255,0.005799,0.046644,0.009756,-0.007373,0.058256,-0.004308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WALLSMATERIAL_MODE_Panel,0.002300,-0.033119,-0.020892,0.032753,0.046644,0.054988,0.049759,0.092652,-0.013597,-0.004193,...,-0.020005,-0.092112,-0.045358,-0.039892,-0.038117,1.000000,-0.270257,-0.069666,0.502903,-0.039280
"WALLSMATERIAL_MODE_Stone, brick",-0.001281,-0.012657,-0.025088,0.016523,0.009756,0.009189,0.012888,0.010782,-0.010964,-0.006327,...,0.027683,-0.091023,-0.044822,-0.039421,-0.037666,-0.270257,1.000000,-0.068843,0.491544,-0.007605
WALLSMATERIAL_MODE_Wooden,-0.000270,0.007946,0.011036,-0.003369,-0.007373,-0.007711,-0.009135,-0.056263,0.008758,-0.002846,...,0.121678,-0.023464,-0.011554,-0.010162,-0.009710,-0.069666,-0.068843,1.000000,0.078964,0.273271
EMERGENCYSTATE_MODE_No,0.000510,-0.042201,-0.038644,0.050174,0.058256,0.068145,0.062878,0.103513,-0.019852,-0.010907,...,0.053146,0.165333,0.076786,0.073260,0.060550,0.502903,0.491544,0.078964,1.000000,-0.090624
